# Enron 2.0: Predicting California's Energy Consumption

## Team Members:

**Names:** John W. Muhs, Corbett Carell
<br>**Emails:** <u0761102,u0502104>@utah.edu</br>
<br>**Github Repository:** [JohnWMuhs/2019-datascience-project](https://github.com/JohnWMuhs/2019-datascience-project "2019 Data Science Project Github Repo")</br>

## Project Objectives

In this project, we present a power consumption prediction method for the [California ISO](http://www.caiso.com/Pages/default.aspx). CAISO is California's largest Balancing Authority and Market Independent System Operator. CAISO holds a number of responsibilities and functions to ensure a competitive, cost-effective, reliabile, and environmentally friendly energy market. For example, CAISO oversees market transactions between power industry stakeholders in California such as Generation Power Plants, Transmission Owners, Renewables Integrators and Customers.  As such, CAISO closely monitors large-scale generation and consumption trends for the State of California. 

By utilizing hourly CAISO load data from the Energy Information Agency (EIA), weather data from the Darksky API. CAISO As of Milestone 1, only these data sets are used, and correlation between weather variables and load has been found, but other data sets may be included to further improve the model. 

<img src="images/CaliforniaISO_PoweringLivesEconomy.jpg">

<img src="images/CaliforniaISO_OpenTransparentMrkt.jpg">

<img src="images/CaliforniaISO_GreeningGridFutureGen.jpg">



<br>**Project Proposal (Submitted Mar. 1st):** The project was introduced via a project proposal available on Google Docs [here](https://docs.google.com/document/d/1i6FB5gmumkx5CnaKLHzKJk8nae0PirpgJDDWm3NkkZ4/edit?usp=sharing "Project Proposal").</br>

Notes from the peer review session are included in the project proposal on the last page. We have also included the project proposal in a pdf included in the Github repository. 

**Things to add from First Milestone:**
1. Weekday vs. Weekend (Done via Pandas)
2. Number of Customer Accounts ([EIA](https://www.eia.gov/opendata/qb.php?category=1718389))
3. Percent Urban Population of California ([BEA](https://apps.bea.gov/API/signup/index.cfm))
4. Gross State Product (GSP) and/or GDP of USA ([BEA](https://apps.bea.gov/API/signup/index.cfm))
5. Split the weather data into categories
6. Try PCA
7. Try a Linear Regression, kNN Regression, and simple perceptron. 
8. Use a mean error to measure accuracy of regression
9. Could use the mean error from the best model to model accuracy on the other models. 



## Data Sources
<br>**Energy Data:** [U.S. Electric System Operating Data](https://www.eia.gov/opendata/qb.php?category=2123635 "EIA API: Electric System Operating Data")</br>
<br>**Weather Data:** [Darksky API](https://darksky.net/dev "Darksky API")</br>

## Project Timeline

\<img src="images/ProjectFlowChart_Proposal.png">

## Feedback from Peer Review Session

# Section 1: Data Import

## Energy Data Import and Formatting

This section includes the code required to import power consumption from the CAISO Balancing Authority (BA) and the entire California region. The California region contains a number of BAs (the largest of which by far is CAISO), however, we decided to import both BA and region data for comparison. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests 
import numpy as np

pd.options.display.float_format = '{:.2f}'.format

The code below allows a user to quickly copy/paste a link from the EIA API website, and provide a title for it in the urls dictionary. The code takes that url and creates a unified dataframe. Therefore, it is incredibly easy to add additional EIA data in the future. 

In [ ]:
# Import EIA API Key 
EIA_api_key = '53e6a63887dc05efe150165fa890f8da'

# Create Dictionary of urls from which we want to pull the EIA API
urls = {'CAISO_HourlyLoad':'http://api.eia.gov/series/?api_key='+EIA_api_key+'&series_id=EBA.CISO-ALL.D.H',
        'California_HourlyLoad':'http://api.eia.gov/series/?api_key='+EIA_api_key+'&series_id=EBA.CAL-ALL.D.H'}


In [ ]:
# Initialize a dataframe
df = pd.DataFrame()

# Setup dummy variable to ensure that a datetime is only generated once
i = 0

for key in urls:
    
    # API pull
    EIAData = requests.get(urls[key])
    
    # Decode from utf-8
    EIAData = EIAData.content.decode("utf-8")
    
    # Load API pull as a dict
    EIAdict = json.loads(EIAData)
    
    # Access the data in EIAdict
    dfEIA = pd.DataFrame.from_dict(EIAdict['series'])
    dfEIA = dfEIA['data'][0]
    
    # Convert clean data to a dataframe
    dfEIA = pd.DataFrame(dfEIA)
    #print(dfEIA[0])
    
    # Find the datetime
    while i != 1:
        df['DateTime'] = pd.to_datetime(dfEIA[0],format='%Y%m%dT%H', errors='ignore')
        df['DateTime'] = df['DateTime'].dt.tz_convert('US/Pacific')
        i += 1
    
    # Create a new column for each type of data (url) pulled from API
    df[str(key)] = dfEIA[1]
    
# Set DateTime as index of the dataframe (tz-aware)
df.index = df['DateTime']

We decided to complete the project in a daily time resolution. This was done mainly to reduce the dataset to a reasonable size. If we kept the dataset at an hourly time resolution, we would have over 32,000 data points! However converting the data down to a daily time resolution provided a very nicely-sized 1362 data points from which to analyze. 

In [ ]:
# Resample hourly data to daily data (summed energy consumption which has units of MWh)
df = df.resample('D').sum()

In [ ]:
# Add a column that determines whether or not a date is a weekday or weekend day.
df['isWeekend'] = np.ones(df.shape[0]).astype(int)
df['isWeekend'] = df['isWeekend'].where((df.index.weekday == 5) | (df.index.weekday == 6),0).astype(int)

#Add index of Month and Day of Year
df['Month'] = df.index.month
df['DayofYear'] = df.index.dayofyear

In [ ]:
# Read in Customer Accounts (pulled from EIA API in file "EIA_API-MonthlyCustomerCount.ipynb")
dfCustomerAccts = pd.read_csv('CustomerAccts.csv')

# Parse DateTime from string to pandas datetime
dfCustomerAccts['DateTime'] = pd.to_datetime(dfCustomerAccts['DateTime'],format='%Y-%m-%d', errors='ignore')

# Set datetime as index
dfCustomerAccts.index = dfCustomerAccts['DateTime']

# Delete extraneous column
dfCustomerAccts = dfCustomerAccts.drop('DateTime',axis=1)

# Localize Timezone to Pacific
dfCustomerAccts = dfCustomerAccts.tz_localize('US/Pacific', level=0)

In [ ]:
df = pd.merge(df,dfCustomerAccts, how='inner', left_index=True, right_index=True)

The final dataframe looks like this:

In [ ]:
df.head()

We used the following code to export this dataframe to a csv.

In [ ]:
#df.to_csv('energydata.csv')

## Weather Data (DarkSky API) Import and Format

The weather data was pulled in using the DarkSky API. The Darksky API requires inputs of latitude, longitude, and datetime to pull corresponding weather data. Darksky API does not seem to allow pulls over a date range, so for this reason, the code to pull over the entire date range required over 6,000 pulls from the API. So as not to do this every time we run the code, a separate code has been developed and included in the repository that demonstrates how DarkSky data was accessed. The code to pull the weather data can be found in "darksky-api.ipynb".

Weather data was pulled from July 1st, 2015 (constraint from EIA hourly data) to December 31st, 2018. All attributes from the DarkSky API pull have been included in the csv, so all filtering and processing will be included in this notebook. 

In [ ]:
# Pull weather data from static csv
dfWeather = pd.read_csv('darksky_data.csv')

#Set dummy variable
i=0

In [ ]:
# To avoid renaming multiple times and pulling an error (we could include a try except here.)
while i != 1:
    #Take the date time (originally included as a string), and format it into a pd datetime
    dfWeather['DateTime'] = pd.to_datetime(dfWeather['time'],format='%m-%d-%Y', errors='ignore')
    
    # Set Datetime as index of the dataframe
    dfWeather.index = dfWeather['DateTime']
    
    # Drop extraneous columns
    dfWeather = dfWeather.drop(['time','DateTime'],axis=1)
    
    # Set TZ to US-Pacific (both dataframes are tz-aware)
    dfWeather = dfWeather.tz_localize('US/Pacific', level=0)
    i += 1


# Section 2: Data Merge and Visualization

In [ ]:
# Merge EIA and Weather dataframes. Inner only keeps the dates from the dfWeather dataframe. 
dfX = pd.merge(df,dfWeather, how='inner', left_index=True, right_index=True)

# Display head of merged dataframe
dfX.head()

In [ ]:
df_pop = pd.read_csv('calpop.csv')
df_di = pd.Series(pd.date_range(start='7/1/2015', end = '12/31/2018', tz = 'US/Pacific'))
df_pop = pd.concat([df_di, df_pop], axis = 1)
df_pop['Datetime'] = df_pop[0]
df_pop.index = df_pop['Datetime']
df_pop.drop(['Date', 'Datetime', 0],axis = 1, inplace = True)

df_pop.info()

In [ ]:
df_gdp = pd.read_csv('bea_gdp.csv')
df_gdp = pd.concat([df_di, df_gdp], axis = 1)
df_gdp['Datetime'] = df_gdp[0]
df_gdp.index = df_gdp['Datetime']
df_gdp.drop(['Date', 'Datetime', 0],axis = 1, inplace = True)

df_gdp.info()

In [ ]:
dfX = pd.concat([dfX, df_gdp, df_pop], axis=1)
dfX.info()

In [ ]:
#dropping na values and dropping California houylr_load since we will be using CASIO_hourlyload data as our depenedent variable for models
dfX = dfX.dropna()
dfX = dfX.drop(['California_HourlyLoad'], axis = 1)
dfX = dfX[dfX['CAISO_HourlyLoad'] > 400000]
dfX.tail()

In [ ]:
fig, (ax0,ax1,ax2) = plt.subplots(nrows=3, figsize=(18, 10))
#start_date = ["2016-01-01","2017-01-01","2018-01-01"]
#end_date = ["2016-12-31","2017-12-31","2018-12-31"]
 
#ax0.set_title('CAISO Consumption 2016 - 2018')
ax0.plot(dfX['CAISO_HourlyLoad'])
ax0.set_xlabel('Date')
ax0.set_ylabel('Consumption (MWh)')
ax1.plot(dfX['temperatureMin'])
ax1.set_xlabel('Date')
ax1.set_ylabel('Min Daily Temperature (Celsius)')
ax2.plot(dfX['dewPoint'])
ax2.set_xlabel('Date')
ax2.set_ylabel('Dew Point (Celsius)')

#df2017.head()

# Section 3: Data Exploration through Modeling

Here we will use a random forest to help us understand which variable are most important to determining load. This will allow us to create a new data frame with fewer variables in order to eliminate uninformative variables from our data. 

In [ ]:
from sklearn.model_selection import train_test_split

training_sample2, testing_sample2 = train_test_split(dfX, test_size=0.3, random_state=42)

print("Training data set size = ", len(training_sample2))
print("Testing data set size = ", len(testing_sample2))

In [ ]:
train_labels = np.array(training_sample2['CAISO_HourlyLoad'])
train_features = training_sample2.drop('CAISO_HourlyLoad', axis=1)

training_sample2 = train_features
train_feature_list = list(train_features.columns)

train_features = np.array(train_features)

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 400, random_state = 42, min_impurity_decrease = 0.01)

rf.fit(train_features, train_labels)

In [ ]:
print('Num of features when fit is perform')
print(rf.n_features_)
print('--------------------')
print('Num of outputs when fit is performed')
print(rf.n_outputs_)
print('--------------------')
print('Feature Importance')
print(rf.feature_importances_)

In [ ]:
from sklearn.model_selection import cross_validate

#Here we are determining the accuracy of our random forest
test_labels = np.array(testing_sample2['CAISO_HourlyLoad'])
test_features = testing_sample2.drop('CAISO_HourlyLoad', axis=1)

testing_sample2 = test_features
test_feature_list = list(test_features.columns)

test_features = np.array(test_features)

accuracy = cross_validate(rf, test_features, test_labels, cv=10)['test_score']
print('The accuracy is: ',sum(accuracy)/len(accuracy)*100,'%')

In [ ]:
# Get numerical feature importances
import matplotlib.pyplot as plt

%matplotlib inline

importances = list(rf.feature_importances_)

# Set the style
plt.style.use('fivethirtyeight')
plt.figure(figsize=(16,8))

# list of x locations for plotting
x_values = list(range(len(importances)))

# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')

# Tick labels for x axis
plt.xticks(x_values, train_feature_list, rotation='vertical')

# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances') 

Here we are using the importance of the variables to create a new dataframe of only the 10 most important variables in our data.

In [ ]:
importances = pd.Series(rf.feature_importances_)
importances
names = pd.Series(train_feature_list)
names
import_df = pd.concat([importances, names], axis=1)
import_df
skim_df = import_df.sort_values(by=[0] , ascending = False) 
skim_df.reset_index(inplace = True)
del skim_df["index"]
thin_df = skim_df.loc[0:9]
print(thin_df)
column_list = list(skim_df.iloc[0:10,1])
column_list.append('CAISO_HourlyLoad')
print(column_list)
df_slim = dfX[column_list]

df_slim.corr()

Here we are using 2 scatter plot matrices to get a visual of the interaction between our different variables.

In [ ]:
df_slim_1 = dfX[['CAISO_HourlyLoad', 
                    'apparentTemperatureMin', 
                    'temperatureMin', 
                    'apparentTemperatureLow', 
                    'isWeekend', 
                    'temperatureLow']]

pd.plotting.scatter_matrix(df_slim_1, figsize=(30, 30), diagonal='kde')
plt.show()

In [ ]:
df_slim_2 = dfX[['CAISO_HourlyLoad',
                 'apparentTemperatureHigh', 
                 'apparentTemperatureMax', 
                 'DayofYear', 
                 'temperatureHigh', 
                 'temperatureMax']]

pd.plotting.scatter_matrix(df_slim_2, figsize=(30, 30), diagonal='kde')
plt.show()

In [ ]:
import statsmodels.formula.api as sm

ols1 = sm.ols(formula="CAISO_HourlyLoad ~ temperatureMin + dewPoint", 
              data=dfX).fit()

ols1.summary()

**Interpretation:** I decided to use the 2 most important variables for our first regression model. We get good results for our first model. Our p-values are very low and the F-stat is very high which are both good signs that the model is statistically significant.

In [ ]:
ols2 = sm.ols(formula="CAISO_HourlyLoad ~ temperatureMin", data=dfX).fit()
ols2.summary()

**Interpretation:** By eliminating the dewPoint from the regression we see a huge increase in the F-stat for the model,  however, the R-squared dropped. These mean that the model is more statistically significant than the previous model but doesn't fit the data as well as the first. 

In [ ]:
ols3 = sm.ols(formula="CAISO_HourlyLoad ~ I(temperatureMin ** 5)", data=dfX).fit()
ols3.summary()

In [ ]:
ols4 = sm.ols(formula="CAISO_HourlyLoad ~ I(temperatureMin ** 5) + dewPoint + isWeekend", data=dfX).fit()
ols4.summary()

In [ ]:
ols4 = sm.ols(formula="CAISO_HourlyLoad ~ I(temperatureMin ** 7)", data=dfX).fit()
ols4.summary()

In [ ]:
ols_test = sm.ols(formula="CAISO_HourlyLoad ~ apparentTemperatureMin + I(temperatureMin ** 3) + dewPoint + isWeekend", 
              data=dfX).fit()

ols_test.summary()

In [ ]:
from sklearn import linear_model
lr = linear_model.LinearRegression()

x = (dfX[['apparentTemperatureMin', 
          'temperatureMin', 
          'apparentTemperatureLow', 
          'isWeekend', 
          'temperatureLow']].values)

y = dfX['CAISO_HourlyLoad'].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
     
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

dif_lr = y_test_lr - y_pred_lr
# print(dif_lr)
dif_lr_s = pd.Series(dif_lr)
print(dif_lr_s.describe())
ypred_mean_lr = np.mean(dif_lr)
load_mean_lr = np.mean(y_test_lr)
print('-------------------------------------')
print(ypred_mean_lr/load_mean_lr)

**Interpretation:** After looking at the scatter plot matrix I noticed that CAISO_HourlyLoad and temperatureMin could have an exponential relationship so I decided to test a few linear models with different exponents. By increasing the exponential power of the temperatureMin variable in the model we are able to increase both the F-stat and R-squared. One thing to note is when we increase the power to 7 is when we start to see the F-stat and R-squared begin to decrease instead of increase. A temperatureMin with a power of 6 is the best linear model I found. 

**Next Steps:** We need to use cross-validation to verify that our model is accurate and that we have not overfitted the model to the data. 

In [ ]:
#PCA
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale 

X = dfX[['apparentTemperatureMin',
         'temperatureMin',
         'dewPoint',
         'isWeekend',
         'apparentTemperatureMax',
         'apparentTemperatureLow',
         'CA_CustomerCount',
         'apparentTemperatureHigh',
         'moonPhase',
         'temperatureLow']]

X_scale = scale(X)

pca_model = PCA()
energy_pca = pca_model.fit_transform(X_scale)

pca_df = pd.DataFrame(energy_pca, columns=['PC1', 'PC2',
                                           'PC3', 'PC4', 'PC5','PC6', 'PC7', 'PC8', 'PC9', 'PC10'])

# Variance ratio of the four principal components
var_ratio = pca_model.explained_variance_ratio_
print(var_ratio)
print('-------------------------------------')

plt.plot(range(len(pca_df.columns)), var_ratio, '-o')

plt.ylabel('Proportion of Variance Explained')
plt.xlabel('Principal Component')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks(range(len(pca_df.columns)))
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap

cmap_ = 'OrRd'
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

y = dfX.CAISO_HourlyLoad

plt.scatter(energy_pca[:, 0], energy_pca[:, 1], c=y, cmap=cmap_, s=30)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

In [ ]:
pca_df.index = dfX.index
dfPCA = pd.concat([dfX, pca_df], axis = 1)
# dfPCA.head()

In [ ]:
ols_test = sm.ols(formula="CAISO_HourlyLoad ~ PC1 + PC2 + PC3 + PC4 + PC5", 
              data=dfPCA).fit()

ols_test.summary()

In [ ]:
from sklearn import linear_model
lr = linear_model.LinearRegression()

x = (dfX[['temperatureMin', 'dewPoint']].values)
y = (dfX['CAISO_HourlyLoad'].values).reshape(-1, 1)

X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(x, y, test_size=0.3, random_state=1)
     
lr.fit(X_train_lr, y_train_lr)
y_pred_lr = lr.predict(X_test_lr)

dif_lr = y_test_lr - y_pred_lr
# print(dif_lr)
dif_lr_s = pd.Series(dif_lr)
print(dif_lr_s.describe())
ypred_mean_lr = np.mean(dif_lr)
load_mean_lr = np.mean(y_test_lr)
print('-------------------------------------')
print(ypred_mean_lr/load_mean_lr)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale 
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error
from sklearn.linear_model import LinearRegression

X = dfX[['apparentTemperatureMin',
         'temperatureMin',
         'dewPoint',
         'isWeekend',
         'apparentTemperatureMax',
         'apparentTemperatureLow',
         'CA_CustomerCount',
         'apparentTemperatureHigh',
         'moonPhase',
         'temperatureLow']]

y = dfX['CAISO_HourlyLoad']

X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=.3, random_state = 1)


In [ ]:
# MLP regression with Scikit-Learn
mlp_reg = MLPRegressor(hidden_layer_sizes=(100,100,100,100,100), 
                       verbose=0, 
                       random_state=2, 
                       solver='adam')
print(mlp_reg.get_params())

mlp_reg.fit(X_train, y_train)
y_pred = mlp_reg.predict(X_test)

# print(mean_squared_error(y_test, y_pred))

print(mlp_reg.score(X_test,y_test)) # score = 1 is good

In [ ]:
dif = y_test - y_pred
dif_s = pd.Series(dif)
print(dif_s.describe())
ypred_mean = abs(np.mean(dif))
load_mean = np.mean(y_test)
print('-------------------------------------')
print(ypred_mean/load_mean)

In [ ]:
# MLP regression with Scikit-Learn
X = pca_df[['PC1',
           'PC2',
           'PC3',
           'PC4',
           'PC5']]

y = dfX['CAISO_HourlyLoad']

X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X, y, test_size=.3, random_state = 1)

mlp_reg = MLPRegressor(hidden_layer_sizes=(100,100,100,100,100), 
                       verbose=0, 
                       random_state=2, 
                       solver='adam')

print(mlp_reg.get_params())

mlp_reg.fit(X_train_pca, y_train_pca)
y_pred_pca = mlp_reg.predict(X_test_pca)

# print(mean_squared_error(y_test_pca, y_pred_p))

print(mlp_reg.score(X_test_pca,y_test_pca)) # score = 1 is good

In [ ]:
dif_pca = y_test_pca - y_pred_pca
dif_pca_s = pd.Series(dif_pca)
print(dif_pca_s.describe())
ypred_mean_pca = abs(np.mean(dif_pca))
load_mean_pca = np.mean(y_test_pca)
print('-------------------------------------')
print(ypred_mean_pca/load_mean_pca)